## Swedcham Debit Note Automation

1. Read American Credit Card Invoice
2. Gather all needed pieces of information
3. Create a new Word File with the current Debit Note
4. Generate a PDF from the Word File
5. Send a mail to Swedcham

#### 1. Read American Credit Card Invoice

In [22]:
# Import Libraries
import PyPDF2
from docxtpl import DocxTemplate
import datetime
import os
import re
import datetime
import locale
import json

In [25]:
# Define Script Variables

# Get American Credit Card Invoice folder
cc_folder = 'D:\\Primeway\\Users Pw\\Oscar.Carvalho\\Contas\\Cartões de Crédito\\American\\'
cc_pass = 'LaylaLaylaLayla'

# Get Destination Folder and Template
nd_folder = 'D:\\Primeway\\Primeway\\Administração\\Contas a Receber\\Nota de Débito\\'
nd_template = 'Primeway - Nota de Débito - Template.docx'
nd_filename = "Primeway - Nota de Débito {} - {}-{}.docx"
nd_recipient = 'elaine@swedcham.com.br'
nd_sender = 'oscar@primeway.com.br'

# Charge parameters
nd_desc_template = "Plano Mensal MAILCHIMP - US$ {} (cotação R$ {} + 6,38% IOF + Comissão 10%)"
nd_IOF = 0.0638
nd_comission = 0.1


In [4]:
# Get last invoice file name

# Get current year
today = datetime.date.today()
year = str(today.year)
# print(year)

path = cc_folder + year + '\\'

file_list = [entry.name for entry in os.scandir(path) if entry.name[0:4] == year]
filename = file_list[len(file_list)-1]

print(path)
print(filename)

D:\Primeway\Users Pw\Oscar.Carvalho\Contas\Cartões de Crédito\American\2023\
2023-07-Extrato-American-Bradesco.pdf


In [5]:
# Open PDF file and get Mailchimp line
filename = '2023-07-Extrato-American-Bradesco.pdf'

with open(path + filename, 'rb') as obj:
   pdfReader = PyPDF2.PdfReader(obj)
   if pdfReader.is_encrypted:
      pdfReader.decrypt(cc_pass)
   page_txt = pdfReader.pages[0].extract_text()

pattern = '[0-9]+/[0-9]+/[0-9]+.+' + 'MAILCHIMP.+' + '\\n'
mailchimp_line = re.search(pattern, page_txt)
if (not mailchimp_line == None):
   mailchimp_line = mailchimp_line.group().replace(',', '.')
   print(mailchimp_line)
else:
   print('Mailchimp line not found!')

19/06/2023 MAILCHIMP 55.00 USD 55.00 5.0300 276.65



#### 2. Gather all needed pieces of information

In [6]:
# Get most of the context variables from CC PDF

locale.setlocale(locale.LC_ALL, 'pt_BR')

items = re.split('[\s|\\n]', mailchimp_line)
nd_date_of_charge = items[0]
nd_value_usd      = items[4]
nd_value_brlusd   = items[5]
nd_value_brl      = items[6]

# Calc totals
nd_amount = round(float(nd_value_usd) * float(nd_value_brlusd) * (1 + nd_IOF) * (1 + nd_comission), 2)
nd_amount = str(nd_amount).replace('.', ',')

# Format values
nd_value_usd    = str(nd_value_usd).replace('.', ',')
nd_value_brlusd = str(round(float(nd_value_brlusd), 2)).replace('.', ',')

nd_description = nd_desc_template.format(nd_value_usd, nd_value_brlusd)
print(nd_description)

# Get current date
nd_current_date = datetime.date.today().strftime("%d de %B de %Y")
print(nd_current_date)


Plano Mensal MAILCHIMP - US$ 55,00 (cotação R$ 5,03 + 6,38% IOF + Comissão 10%)
28 de julho de 2023


In [9]:
# Get due date

# Get date object from date of charge
date_obj = datetime.datetime.strptime(nd_date_of_charge, '%d/%m/%Y')
print(date_obj)
# # Calc due date
nd_due_date = date_obj.replace(day=1) + datetime.timedelta(days=31*2)
nd_due_date = nd_due_date - datetime.timedelta(days=nd_due_date.day)

# Format Due Date
nd_due_date = nd_due_date.strftime("%d/%m/%Y")
print(nd_due_date)


2023-06-19 00:00:00
31/07/2023


In [15]:
# Get sequence of the Nota de Débito

nd_sequence = ''

# Get filename
file_list = [entry.name for entry in os.scandir(nd_folder) if entry.name[-3:] == 'pdf']
filename = file_list[len(file_list)-1]
print(filename)

# Calc new sequence
pattern = '0[0-9]+(?=\s)'
last_sequence = re.search(pattern, filename)

nd_sequence = int(last_sequence.group()) + 1
nd_sequence = str(nd_sequence).zfill(4)
print(nd_sequence)

Primeway - Nota de Débito 0032 - 2023-07.pdf
0033


In [20]:
# Prepare context

context = { 
   'nd_current_date'   : nd_current_date,
   'nd_sequence'       : nd_sequence,
   'nd_recipient'      : nd_recipient,
   'nd_description'    : nd_description,
   'nd_date_of_charge' : nd_date_of_charge,
   'nd_amount'         : nd_amount,
   'nd_due_date'       : nd_due_date
}
pretty_context = json.dumps(context, indent=4)

print(pretty_context)

{
    "nd_current_date": "28 de julho de 2023",
    "nd_sequence": "0033",
    "nd_recipient": "elaine@swedcham.com.br",
    "nd_description": "Plano Mensal MAILCHIMP - US$ 55,00 (cota\u00e7\u00e3o R$ 5,03 + 6,38% IOF + Comiss\u00e3o 10%)",
    "nd_date_of_charge": "19/06/2023",
    "nd_amount": "323,73",
    "nd_due_date": "31/07/2023"
}


#### 3. Create a new Word File with the current Debit Note

In [26]:
# Open template and render
doc = DocxTemplate(nd_folder + nd_template)
doc.render(context)

# Save a new Nota de Débito
nd_filename = nd_filename.format(nd_sequence, nd_due_date[-4:], nd_due_date[3:5])
doc.save(nd_folder + nd_filename)